Document Search Engine:
---------------------------
Dalam latihan kali ini kita akan membangun "documents search engine" dengan menggunakan "20newsgroup open-source dataset"
Github source: https://github.com/zayedrais/DocumentSearchEngine
Dataset source: http://qwone.com/~jason/20Newsgroups/

Import required libraries

In [77]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer


Data loading dari GitHub. Simpan di variabel "news"

In [78]:
news = pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/newsgroups.json')

Screening variabel "news" untuk membuat dataframe dengan dua kolom: subject dan content

In [79]:
for i,txt in enumerate(news['content']):
    #print(i)
    subject = re.findall('Subject:(.*\n)',txt)
    if (len(subject) !=0):
        news.loc[i,'Subject'] =str(i)+' '+subject[0]
    else:
        news.loc[i,'Subject'] ='NA'


In [80]:
df_news =news[['Subject','content']]

In [81]:
df_news.head(5)

,Subject,content
0,0 WHAT car is this!?\n,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,1 SI Clock Poll - Final Call\n,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,2 PB questions...\n,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,3 Re: Weitek P9000 ?\n,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,4 Re: Shuttle Launch Question\n,From: jcm@head-cfa.harvard.edu (Jonathan McDow...


Buat hurus kecil semua untuk content

In [82]:
df_news['content']=[entry.lower() for entry in df_news['content']]

<ipython-input-82-e54f969b1a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['content']=[entry.lower() for entry in df_news['content']]


Hapus beberapa kata berulang

In [83]:
SW=['subject:','organization:','thanks','thank','re:']


In [84]:
for sw in SW:
    df_news.content=df_news.content.replace(to_replace=sw,value='',regex=True)


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Bersihkan data untuk beberapa karakter khusus dengan menggantinya menjadi '' atau kosong

In [85]:
## data Cleaning for content data of news
df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True) #remove from to email 
df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_news.content =df_news.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_news.content =df_news.content.replace(to_replace='-',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.content =df_news.content.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_news.content =df_news.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace


In [86]:
## data cleaning for subject data 
df_news.Subject =df_news.Subject.replace(to_replace='Re:',value='',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_news.Subject =df_news.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.Subject =df_news.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
df_news.Subject =df_news.Subject.apply(lambda x:x.strip())

Drop content yang kosong

In [87]:
## ## Checking  and drop empty data
for i,sen in enumerate(df_news.content):
    if len(sen.strip()) ==0:
        print(str(i))
        #file_data.text[i] = np.nan
        df_news=df_news.drop(str(i),axis=0).reset_index().drop('index',axis=1)

Downliad tools yang digunakan untuk tokenisasi

In [88]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [89]:
# okenization : In this each entry in the file_data will be broken into set of words
df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]


<ipython-input-89-4b719d01c813>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]


Download tools dan membersihkan data lebih lanjut: Membuang stop words, lemmatisasi (diubah menjadi akar kata)

In [90]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
df_news.shape

(11314, 3)

Kata kata stopwords dalam bahasa Inggris

In [93]:
#print(stopwords.words('english'))


In [94]:
df_news.head(2)

,Subject,content,Word tokenize
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,..."


Proses lematisasi

In [95]:
#df_news = df_news[0:1000]

In [96]:
## This Function took around 13 hours for word Lemmatized and remove the Stop words & single character of word of each 11314 rows.
df_clean = wordLemmatizer(df_news['Word tokenize'][0:1000]) 
df_clean

,Keyword_final
0,"['car', 'nntp', 'post', 'host', 'university', ..."
1,"['si', 'clock', 'poll', 'final', 'call', 'summ..."
2,"['pb', 'question', 'purdue', 'university', 'en..."
3,"['weitek', 'harris', 'computer', 'system', 'di..."
4,"['shuttle', 'launch', 'question', 'smithsonian..."
...,...
995,"['terminal', 'sale', 'nyx', 'public', 'access'..."
996,"['remember', 'name', 'come', 'election', 'time..."
997,"['interest', 'adb', 'behaviour', 'nntp', 'post..."
998,"['request', 'information', 'essential', 'tremo..."


Membersihkan data lebih lanjut

In [97]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [98]:
## Insert New column in df_news to stored the Clean Keyword
#df_news.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

In [99]:
#df_news.head(10)

Gunakan lemma yang sudah ada (download)

In [100]:
## Already Word lemmatize clean keywords the below path:-

#df_news[['Subject','Clean_Keyword']].to_json("WordLemmatize20NewsGroup.json")
df =pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/WordLemmatize20NewsGroup.json')
df_news['Clean_Keyword'] =df['Clean_Keyword']

In [121]:
df_news.head(5)

,Subject,content,Word tokenize,Clean_Keyword
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...,"[what, car, is, this, nntp, posting, host, rac...","thing,car,nntp,post,host,university,maryland,c..."
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...,"[si, clock, poll, final, call, summary, final,...","guy,kuo,si,clock,poll,final,call,summary,final..."
2,2 PB questions,pb questions... purdue university engineering ...,"[pb, questions, ..., purdue, university, engin...","thomas,willis,pb,question,purdue,university,en..."
3,3 Weitek P9000,weitek p9000 harris computer systems division ...,"[weitek, p9000, harris, computer, systems, div...","joe,green,weitek,harris,computer,system,divisi..."
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...,"[shuttle, launch, question, smithsonian, astro...","jonathan,mcdowell,shuttle,launch,question,smit..."


In [101]:
df_news_save= df_news
df_news_save = df_news_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_news_save

,Subject,content
0,0 WHAT car is this,what car is this nntp posting host rac3.wam.um...
1,1 SI Clock Poll Final Call,si clock poll final call summary final call fo...
2,2 PB questions,pb questions... purdue university engineering ...
3,3 Weitek P9000,weitek p9000 harris computer systems division ...
4,4 Shuttle Launch Question,shuttle launch question smithsonian astrophysi...
...,...,...
11309,11309 Migraines and scans,migraines and scans distribution world inventi...
11310,11310 Screen Death Mac Plus 512,screen death mac plus 512 tufts university med...
11311,11311 Mounting CPU Cooler in vertical case,mounting cpu cooler in vertical case mail grou...
11312,11312 Sphere from 4 points,sphere from 4 points central research lab. hit...


In [102]:
df_news_save.to_csv("df_news_index.csv", index=False, header=True)

In [103]:
## Sample of clean keywords
df_news.Clean_Keyword[0]

'thing,car,nntp,post,host,university,maryland,college,park,line,wonder,anyone,could,enlighten,car,saw,day,sport,car,look,late,early,call,bricklin,door,really,small,addition,front,bumper,separate,rest,body,know,anyone,tellme,model,name,engine,spec,year,production,car,make,history,whatever,info,funky,look,car,please,mail,il,bring,neighborhood,lerxst'

Persiapan untuk membangun "tF-iDF", membangun vocabularly

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df_news.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

# Fit the TfIdf model
tfidf.fit(df_news.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_news.Clean_Keyword)


In [105]:
vocabulary[0:4]

['mencken', 'cinsider', 'airhead', 'gayri']

In [106]:
### Save model
with open('/content/tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [107]:
### load model
t = pickle.load(open('/content/tfid.pkl','rb'))

In [108]:
### Save Vacabulary
with open("/content/vocabulary_news20group.txt", "w") as file:
    file.write(str(vocabulary))


In [109]:
### load Vacabulary
with open("/content/vocabulary_news20group.txt", "r") as file:
    data2 = eval(file.readline())

In [110]:
print(data2[0:10])

['mencken', 'cinsider', 'airhead', 'gayri', 'lesseee', 'celibate', 'upjohn', 'sro', 'hitti', 'gaze']


In [111]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):

    Q = np.zeros((len(vocabulary)))
    
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q


In [112]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim


In [113]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    #print("\nQuery:", query)
    #print("")
    #print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in tfidf_tran.A:
        
        d_cosines.append(cosine_sim(query_vector, d))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_news['Subject'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [114]:
%time cosine_similarity_T(10,'computer science')
#df_news['Subject']


CPU times: user 3.26 s, sys: 3.95 s, total: 7.2 s
Wall time: 4.65 s


,index,Subject,Score
0,2231,2231 Computer Engr vs Computer Science,0.396137
1,10340,10340 Science and Methodology,0.302603
2,4173,4173 Rawlins debunks creationism,0.284936
3,4326,4326 Computer Engr vs Computer Science,0.284256
4,6921,6921 Automatic layout of state diagrams,0.260732
5,7618,7618 Solution Why do I need xrdb m when Xdefau...,0.251170
6,5741,5741 Rawlins debunks creationism,0.247442
7,8464,8464 Date is stuck,0.238985
8,3177,3177 Homeopathy a respectable medical tradition,0.235738
9,4366,4366 Computer Engr vs Computer Science,0.227186


In [122]:
%time cosine_similarity_T(5,'power supply')

CPU times: user 3.35 s, sys: 5.06 s, total: 8.41 s
Wall time: 5.83 s


,index,Subject,Score
0,3948,3948 Switch mode power supply,0.577529
1,6799,6799 Choosing an appropriate Power Supply PLEA...,0.409456
2,7445,7445 PACE MODEM,0.361670
3,2579,2579 Original IBM PC specs,0.343964
4,4555,4555 Bare 4mb 386 25 Micronics system 450 350 ...,0.318024


In [115]:
df_news.content[2231]

'computer engr vs. computer science distribution usa ncr microelectronics products division an at t company in article 1993apr10.210242.340 macadam.com mike macadam.com writes i am a freshman in college and can t decide whether to major in computer engineering or computer science. any advice or suggestions will be appreciated. mike a professor of mine once said the difference between a computer engineer and a computer scientist is about 5000 meaning the engineer makes 5000 more than the cs. seriously though the main difference is that most cs people write programs that people will use i.e. database graphics word processors etc. while an engineer writes for machines or control systems i.e. the computer in your car a flight control system computer controled devices etc. in other words cs writes software while cse writes firmware. these are generalizations but for the most part that is what the difference is. p.s. the 5000 is not just a joke scott'

In [116]:
!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 20 kB/s 
     |████████████████████████████████| 1.7 MB 43.7 MB/s 
     |████████████████████████████████| 6.0 MB 43.3 MB/s 
     |████████████████████████████████| 439 kB 62.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [117]:
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel


In [118]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.11.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [119]:
#!mkdir /home/zettadevs/GoogleUSE Model
#! curl -L -o 4.tar.gz "https://tfhub.dev/google/universal-sentence-encoder/4?tf-hub-format=compressed" 
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
#module_path ="/home/zettadevs/GoogleUSEModel/USE_4"
%time model = hub.load(module_url)
#print ("module %s loaded" % module_url)

#Create function for using modeltraining
def embed(input):
    return model(input)



CPU times: user 12.8 s, sys: 5.69 s, total: 18.5 s
Wall time: 47.4 s
